In [263]:
import thefuzz.fuzz
from thefuzz import process

Простая проверка равенства двух строк

In [264]:
str_A = 'Привет Мир'
str_B = 'Привет миру мир'
print(str_A == str_B)

False


In [265]:
str_input = input() # ввод строки пользователем
if str_A == str_input:
    print("Строки одинаковые")
else :
    print("Строки разные")

print("Строки одинаковые" if str_A == str_input else "Строки разные") # тернарный оператор if else
print("Строка " + str_A + (" равна " if str_A == str_input else " не равна ") + str_input )
print(f"Строка \"{str_A}\" %s \"{str_input}\""%(" равна " if str_A == str_input else " не равна "))

Строки разные
Строки разные
Строка Привет Мир не равна 1
Строка "Привет Мир"  не равна  "1"


Почему одинаковые по буквам строки могут быть не равны?

In [266]:
print(f"Строка \"{str_A.lower()}\"  %s \"{str_input.lower()}\""%(" равна " if str_A == str_input else " не равна ")) # нижний регистр
print(f"Строка \"{str_A.upper()}\" %s \"{str_input.upper()}\""%(" равна " if str_A == str_input else " не равна ")) # верхний регистр
print(f"Строка \"{str_A}\" %s \"{str_input}\""%(" равна " if str_A.lower() == str_input.lower() else " не равна ")) # Исходные строки


Строка "привет мир"   не равна  "1"
Строка "ПРИВЕТ МИР"  не равна  "1"
Строка "Привет Мир"  не равна  "1"


А как оценить процентное соотношение равенства двух строк?
Функция thefuzz.fuzz.ratio()

In [289]:
a = thefuzz.fuzz.ratio(str_A , str_B)
print(a,"%")

33 %


Формула определения Равенства строк fuzz.ratio(str_A,str_B)
        2*кол-во совпадений/сумму длин двух строк

In [268]:
def distance(a, b):
    "Вычисление расстояния между строками алгоритм Левенштейна"
    n, m = len(a), len(b)
    if n > m:
        a, b = b, a
        n, m = m, n
    current_row = range(n + 1)
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            if a[j - 1] != b[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)
    return current_row[n]
distance_levenstain = distance(str_A, str_B)
distance_levenstain_lower = distance(str_A.lower(), str_B.lower())
print("Расстояние между строками -",distance_levenstain)
print("Расстояние между строками -",distance_levenstain_lower)

Расстояние между строками - 6
Расстояние между строками - 5


In [396]:
def leven_ratio(a,b):
    "Расчет в процентах аналог thefuzz.fuzz.ratio()"
    print("Расстояние между строк - " +str(distance(a, b)))
    return str((2*(max(len(a),len(b))-distance(a, b))/(len(a + b)))*100)
print(leven_ratio(str_A,str_B),"%")
print(leven_ratio(str_A.lower(), str_B.lower()),"%")

Расстояние между строк - 16
21.052631578947366 %
Расстояние между строк - 16
21.052631578947366 %


Функция thefuzz.fuzz.partial_ratio()

In [412]:
str_A = 'Привет мир'
str_B = 'Приве1'
str_A = 'мир наш привет'
str_B = 'мир наш привет мой'
thefuzz.fuzz.ratio(str_A , str_B)
print("Обычное сравнение",thefuzz.fuzz.ratio(str_A , str_B),"%")
print("Частичное сходство строк",thefuzz.fuzz.partial_ratio(str_A , str_B),"%")

Обычное сравнение 88 %
Частичное сходство строк 100 %


In [345]:
def part_ratio(a,b):   # может это дать на самостоятельную, кастрированная версия поиска подстроки в строке?? либо дать ее доделать чтобы в итоге получить процент при том что смотреть пример выше!
    "Очень простой аналог функции thefuzz.fuzz.partial_ratio"
    if len(a) >= len(b):
        short, long = b,a
    else:
        short, long = a,b
    for i in range(len(long)-len(short)+1) :
        if short == long[i:(i+len(short))]:
            return True
        else :
            continue
str_A = 'Привет мир'
str_B = 'мир'
print(part_ratio(str_A , str_B))

True


Функция thefuzz.fuzz.token_sort_ratio()

In [374]:
str_A = 'Привет     мир наш'
str_B = 'наш мир привет'
print("Обычное сравнение",thefuzz.fuzz.ratio(str_A , str_B),"%")
print("Частичное сходство строк",thefuzz.fuzz.partial_ratio(str_A , str_B),"%")
print("Сравнение в разном порядке",thefuzz.fuzz.token_sort_ratio(str_A , str_B),"%")

Обычное сравнение 31 %
Частичное сходство строк 36 %
Сравнение в разном порядке 100 %


In [375]:
def token_sort(a,b):
    "Расчет в процентах аналог thefuzz.fuzz.token_sort_ratio()"
    sort_a = sorted(a.lower().split())
    sort_b =  sorted(b.lower().split())
    a = " ".join(sort_a)
    b = " ".join(sort_b)
    print(a,b)
    if len(a) == len(b):
        return leven_ratio(a,b)
    else:
        return ("в разработке") # тут должна быть доделанная версия def part_ratio(a,b):
print(token_sort(str_A,str_B))

мир наш привет мир наш привет
Расстояние между строк - 0
100.0 %


Функция thefuzz.fuzz.token_set_ratio()

In [423]:
str_A = 'mariners vs angels'
str_B = 'los angeles angels of anaheim at seattle mariners'
print("Обычное сравнение",thefuzz.fuzz.ratio(str_A , str_B),"%")
print("Частичное сходство строк",thefuzz.fuzz.partial_ratio(str_A , str_B),"%")
print("Сравнение в разном порядке",thefuzz.fuzz.token_sort_ratio(str_A , str_B),"%")
print("Сравнение наборов токенов",thefuzz.fuzz.token_set_ratio(str_A , str_B),"%")

Обычное сравнение 36 %
Частичное сходство строк 61 %
Сравнение в разном порядке 51 %
Сравнение наборов токенов 91 %


In [425]:
def token_set_ratio(a,b):
    "Расчет в процентах аналог thefuzz.fuzz.token_set_ratio()"
    plenty_sort_a = set(str_A.lower().split())
    plenty_sort_b = set(str_B.lower().split())
    intersection_plenty= " ".join(sorted(plenty_sort_a.intersection(plenty_sort_b)))
    difference_plently = " ".join(sorted(plenty_sort_a.difference(plenty_sort_b)))
    difference_plently2 = " ".join(sorted(plenty_sort_b.difference(plenty_sort_a)))
    combination1 = intersection_plenty + ' ' + difference_plently
    combination2 = intersection_plenty + ' ' + difference_plently2
    pairwise = [
        leven_ratio(intersection_plenty, combination1),
        leven_ratio(intersection_plenty, combination2),
        leven_ratio(combination1, combination2)
       ]
    return max(pairwise)

token_set_ratio(str_A,str_B)

Расстояние между строк - 3
Расстояние между строк - 34
Расстояние между строк - 32


'90.9090909090909'

In [351]:
a = thefuzz.fuzz.WRatio(str_A.lower() , str_B.lower())
print(str(a)+" %")
list_of_strings = ['Привет', "Приветствую", "Приветули"]
process.extract(str_B, list_of_strings,scorer=thefuzz.fuzz.ratio)

95 %


[('Привет', 60), ('Приветули', 52), ('Приветствую', 48)]

In [352]:
print(set(str_A.split()), set(str_B.split()))

{'мой', 'мир', 'Привет', 'наш'} {'мир', 'Привет', 'наш'}


In [355]:
set(str_A.split()).difference(set(str_B.split()))

{'Привет', 'мой'}

In [353]:
set(str_A.split()).intersection(set(str_B.split()))


{'Привет', 'мир', 'наш'}

In [384]:
str_A.strip()

'Привет  мой  мир наш'